# DPK Demo 1

Showcase DPK's capabilities

## Before Running the notebook

Please complete [setting up python dev environment](../setup-python-dev-env.md)

## Step-1: Inspect the Data

We will use simple PDFs about Solar system.  The files are here [../data/solar-system/](../data/solar-system/)

### How to re-generate PDFs?

If you like to play around with various inputs, follow these steps to re-generate PDFs.

First edit the markdown files using any text editor.

Then use [pandoc](https://pandoc.org/) to convert them to pdfs.

```bash
pandoc  earth.md  -o earth.pdf
pandoc  mars.md  -o mars.pdf
```

## Step-2: Configuration

### 2.1 - Basic Config

In [1]:
import os 

## Configuration
class MyConfig:
    pass 

MY_CONFIG = MyConfig ()

## Input Data - configure this to the folder we want to process
MY_CONFIG.INPUT_DATA_DIR = os.path.join (os.path.abspath (''), '..', 'data', 'solar-system')
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  1
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

In [2]:
## Add parent dir to path
import os,sys 

this_dir = os.path.abspath('')
parent_dir = os.path.dirname(this_dir)
sys.path.append (os.path.abspath (parent_dir))

### 2.2 - Setup input/outpur directories

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

STAGE = 0

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


## Step-3: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 3.1 - Set Input/output Folder

In [4]:
STAGE  += 1
# STAGE = 1  ## DEBUG

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_parquet_out")

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-examples/demo-1/../data/solar-system' --> output='output/01_parquet_out'


### 3.2 - Execute 

In [5]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from data_processing_ray.runtime.ray import RayTransformLauncher
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


01:45:39 INFO - Running locally
01:45:39 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': False}
01:45:39 INFO - data factory data_ is using local data access: input_folder - /home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-examples/demo-1/../data/solar-system output_folder - output/01_parquet_out
01:45:39 INFO - data factory data_ max_files -1, n_sample -1
01:45:39 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
01:45:39 INFO - pipeline id pipeline_id
01:45:39 INFO - code location None
01:45:39 INFO - number of workers 2 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
01:45:39 INFO - actor creation delay 0
01:45:39 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job

✅ Stage:1 completed successfully
CPU times: user 3.95 s, sys: 1.3 s, total: 5.25 s
Wall time: 39 s


### 3.3 - Inspect Generated output

Here we should see one entry per input file processed.

In [6]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (2, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,mars.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",1,0,11,32038943-af7b-43fd-b5e9-c640fdb37c54,pdf,5556be7a44be95d5f40a5235fe1e8774367a070b20eef6...,2505,2024-09-06T01:46:03.416974,1.689363,mars.pdf
1,earth.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf



### 3.4 - Understand the output

Here are some interesting attributes to note:

- **filename** : original filename
- **contents** : text
- **document_id**: unique id (UUID) assignd to this document
- **hash** : hash of document (TODO: what kind of hash?)
- **pdf_convert_time** : time to convert this pdf in seconds

Let's inspect the **contents** column.  See how the text is being divided up!

In [7]:
import pprint
import json

pprint.pprint (json.loads(output_df.iloc[0, ]['contents']), indent=4)

{   '_name': '',
    'description': {'logs': []},
    'equations': [],
    'figures': [],
    'file-info': {   '#-pages': 1,
                     'document-hash': '0ff74fc417f3c08a05fa8b4d631b898b32a777c01c109391405ee5324d54b9cb',
                     'filename': 'mars.pdf',
                     'page-hashes': [   {   'hash': '5ae83e3b85582da642213d2e8a41b382a8a2c24c88f2b2a76a96ba0725493ecf',
                                            'model': 'default',
                                            'page': 1}]},
    'footnotes': [],
    'main-text': [   {   'name': 'Section-header',
                         'prov': [   {   'bbox': [   133.35,
                                                     657.12,
                                                     169.62,
                                                     667.98],
                                         'page': 1,
                                         'span': [0, 4]}],
                         'text': 'Mars',
             

##  Step-4: Doc chunks

In the previous step, we have extracted text from oru PDFs.  But we have the content of entire file as 'one row' in our parquet output.

In this step, we are going to split the documents in chunks, according to their layout segmentation.

This transform uses [Quackling](https://github.com/DS4SD/quackling) `HierarchicalChunker`
to chunk according to the document layout segmentation, i.e. respecting the original document components as paragraphs, tables, enumerations, etc.
It relies on documents converted with the Docling library in the [pdf2parquet transform](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/pdf2parquet/python/README.md) using the option `contents_type: "application/json"`,
which provides the required JSON structure.

### 4.1 - Set Input/output Folder

In [8]:
STAGE  += 1
# STAGE = 2  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_chunk_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### 4.2 - Execute 

In [9]:
%%time 

from data_processing_ray.runtime.ray import RayTransformLauncher
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/sujee/apps/anaconda3/envs/data-prep-
[nltk_data]     kit-3-py311/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!
01:46:15 INFO - Running locally
01:46:15 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
01:46:15 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
01:46:15 INFO - data factory data_ max_files -1, n_sample -1
01:46:15 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
01:46:15 INFO - pipeline id pi

✅ Stage:2 completed successfully
CPU times: user 1.35 s, sys: 935 ms, total: 2.28 s
Wall time: 18.2 s


### 4.3 - Inspect Generated output

We would see documents are split into many chunks

In [10]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.head(10)

Files processed : 2
Chunks created : 6
Input data dimensions (rows x columns)=  (2, 12)
Output data dimensions (rows x columns)=  (6, 15)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
0,mars.pdf,1,0,11,32038943-af7b-43fd-b5e9-c640fdb37c54,pdf,5556be7a44be95d5f40a5235fe1e8774367a070b20eef6...,2505,2024-09-06T01:46:03.416974,1.689363,mars.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.82, 590.31, 478.56, 623.02]"
1,mars.pdf,1,0,11,32038943-af7b-43fd-b5e9-c640fdb37c54,pdf,5556be7a44be95d5f40a5235fe1e8774367a070b20eef6...,2505,2024-09-06T01:46:03.416974,1.689363,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is...",$.main-text[5],1,"[132.86, 500.93, 477.3, 534.53]"
2,mars.pdf,1,0,11,32038943-af7b-43fd-b5e9-c640fdb37c54,pdf,5556be7a44be95d5f40a5235fe1e8774367a070b20eef6...,2505,2024-09-06T01:46:03.416974,1.689363,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[7],1,"[147.72, 452.7, 476.98, 474.97]"
3,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.82, 590.31, 478.58, 623.05]"
4,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.89, 512.46, 477.42, 534.47]"
5,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Earth\n· Distance from the Sun: Average of 149...,$.main-text[7],1,"[147.68, 464.66, 477.03, 487.02]"


### 4.4 - Understanding the Output

Here we see 2 PDF files are split into 6 chunks.  Basically we see the documents are being split along 'natural boundaris' - paragraphs and bullet points

See how **document_id** is carried throughout.  This helps us identify original documents.

Also note **contents** is now plain text (not JSON as before)

In [11]:
output_df[['filename', 'contents']]

,filename,contents
0,mars.pdf,Solar System\nOur solar system is a vast and f...
1,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is..."
2,mars.pdf,Basic facts about Mars:\n· Distance from the S...
3,earth.pdf,Solar System\nOur solar system is a vast and f...
4,earth.pdf,Earth\nEarth is the third planet from the Sun....
5,earth.pdf,Earth\n· Distance from the Sun: Average of 149...


In [12]:
print (output_df.iloc[-1]['contents'])

Earth
· Distance from the Sun: Average of 149.6 million kilometers (93 million miles)
· Rotation Period: 24 hours (one day)


## Step-5: Exact Dedup



### 5.1 - Set Input/output Folder

In [13]:
STAGE  += 1
# STAGE  = 3  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_ededupe_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_ededupe_out'


### 5.2 - Execute 

In [14]:
%%time

from data_processing_ray.runtime.ray import RayTransformLauncher
from ededup_transform_ray import EdedupRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

01:46:33 INFO - Running locally
01:46:33 INFO - exact dedup params are {'doc_column': 'contents', 'hash_cpu': 0.5, 'num_hashes': 2}
01:46:33 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_ededupe_out
01:46:33 INFO - data factory data_ max_files -1, n_sample -1
01:46:33 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
01:46:33 INFO - pipeline id pipeline_id
01:46:33 INFO - code location None
01:46:33 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
01:46:33 INFO - actor creation delay 0
01:46:33 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
2024-09-06 01:46:34,977	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orchestrate pid=256451) 01:46:35 INFO - orchestrator s

✅ Stage:3 completed successfully
CPU times: user 96.4 ms, sys: 148 ms, total: 244 ms
Wall time: 14.9 s


### 5.3 - Inspect Generated output

In [15]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.head(10)

Input data dimensions (rows x columns)=  (6, 15)
Output data dimensions (rows x columns)=  (5, 15)
Input chunks before exact dedupe : 6
Output chunks after exact dedupe : 5
Duplicate chunks removed :   1


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
0,mars.pdf,1,0,11,32038943-af7b-43fd-b5e9-c640fdb37c54,pdf,5556be7a44be95d5f40a5235fe1e8774367a070b20eef6...,2505,2024-09-06T01:46:03.416974,1.689363,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is...",$.main-text[5],1,"[132.86, 500.93, 477.3, 534.53]"
1,mars.pdf,1,0,11,32038943-af7b-43fd-b5e9-c640fdb37c54,pdf,5556be7a44be95d5f40a5235fe1e8774367a070b20eef6...,2505,2024-09-06T01:46:03.416974,1.689363,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[7],1,"[147.72, 452.7, 476.98, 474.97]"
2,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.82, 590.31, 478.58, 623.05]"
3,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.89, 512.46, 477.42, 534.47]"
4,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Earth\n· Distance from the Sun: Average of 149...,$.main-text[7],1,"[147.68, 464.66, 477.03, 487.02]"


### 5.4 - Understanding the output

Remember we had 6 chunks initially.  Now we have 5!  One duplicate chunk is removed.

If you look at the PDF, the following common paragraph in `earth.pdf` and `mars.pdf`  is removed from one of the documents!  Pretty neat, eh!

```text
## Solar System

Our solar system is a vast and fascinating expanse, comprising eight planets, five dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.
```

## Step-6:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set **hash_column** to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set **int_id_column** to the name of the column, where you want to store it. 
 
**This is a pre-requisite for fuzzy dedup** in the pipeline.

### 6.1 - Set Input/output Folder

In [16]:

# Input for this stage is the output of exact dedeup component
# output of this component makes it possible for fdedup component to run on data.

STAGE  += 1
# STAGE  = 4  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_doc_id_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output/03_ededupe_out' --> output='output/04_doc_id_out'


### 6.2 - Execute

In [17]:
%%time 

from data_processing_ray.runtime.ray import RayTransformLauncher
from doc_id_transform_ray import DocIDRayTransformConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "hash_column",
    "doc_id_int_column": "int_id_column",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

01:46:48 INFO - Running locally
01:46:48 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'hash_column', 'int_column': 'int_id_column'}
01:46:48 INFO - data factory data_ is using local data access: input_folder - output/03_ededupe_out output_folder - output/04_doc_id_out
01:46:48 INFO - data factory data_ max_files -1, n_sample -1
01:46:48 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
01:46:48 INFO - pipeline id pipeline_id
01:46:48 INFO - code location None
01:46:48 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
01:46:48 INFO - actor creation delay 0
01:46:48 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
2024-09-06 01:46:50,050	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orchestrate pid=258443) 0

✅ Stage:4 completed successfully
CPU times: user 107 ms, sys: 130 ms, total: 238 ms
Wall time: 15.3 s


### 6.3 - Inspect Generated output

You will notice we have two extra columns

- **hash_column**
- **int_id_column**

But still the same number or rows as before

In [18]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.head(10)

Input data dimensions (rows x columns)=  (5, 15)
Output data dimensions (rows x columns)=  (5, 17)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,hash_column,int_id_column
0,mars.pdf,1,0,11,32038943-af7b-43fd-b5e9-c640fdb37c54,pdf,5556be7a44be95d5f40a5235fe1e8774367a070b20eef6...,2505,2024-09-06T01:46:03.416974,1.689363,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is...",$.main-text[5],1,"[132.86, 500.93, 477.3, 534.53]",a31663e06fac41470ecc459f5a58658a3f9997d7801053...,4
1,mars.pdf,1,0,11,32038943-af7b-43fd-b5e9-c640fdb37c54,pdf,5556be7a44be95d5f40a5235fe1e8774367a070b20eef6...,2505,2024-09-06T01:46:03.416974,1.689363,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[7],1,"[147.72, 452.7, 476.98, 474.97]",7ff317954ec5f3b15607c053c30c2b0db0f6b64cc3295a...,5
2,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.82, 590.31, 478.58, 623.05]",44c6e373258c7cdc03f75a8e96a9b160f9aa4e4baf5674...,1
3,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.89, 512.46, 477.42, 534.47]",7c4a750e2215f231803a6f8078bde1e9699034fb033dd3...,2
4,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Earth\n· Distance from the Sun: Average of 149...,$.main-text[7],1,"[147.68, 464.66, 477.03, 487.02]",0337be5c940b3df316e391ae3dc20379442257498812e4...,3


## Step-7: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with the goal of removing code files that may have **slight variations** and thereby unbiasing
the data further. 

Small variations are quite commonly seen in code data in the form of variations in the values of variables, addittion of logging statements etc.

### 7.1 - Set Input/output Folder

In [19]:
## Input to this component is the output of doc_id generator component. 

STAGE  += 1
# STAGE  = 5  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_fdedupe_out")
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output/04_doc_id_out' --> output='output/05_fdedupe_out'


### 7.2 - Execute 

In [20]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration
from data_processing_ray.runtime.ray import RayTransformLauncher

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "int_id_column",
    "fdedup_cluster_column": "hash_column",
    # infrastructure
    "fdedup_bucket_cpu": 0.5,
    "fdedup_doc_cpu": 0.5,
    "fdedup_mhash_cpu": 0.5,
    "fdedup_num_doc_actors": 2,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 2,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.8,
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

01:47:03 INFO - Running locally
01:47:03 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'int_id_column', 'cluster_column': 'hash_column', 'bucket_cpu': 0.5, 'mhash_cpu': 0.5, 'doc_cpu': 0.5, 'num_doc_actors': 2, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 2, 'num_permutations': 64, 'threshold': 0.8, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
01:47:03 INFO - data factory data_ is using local data access: input_folder - output/04_doc_id_out output_folder - output/05_fdedupe_out
01:47:03 INFO - data factory data_ max_files -1, n_sample -1
01:47:03 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
01:47:03 INFO - pipeline id pipeline_id
01:47:03 INFO - code location None
01:47:03 INFO - number of wor

✅ Stage:5 completed successfully
CPU times: user 189 ms, sys: 184 ms, total: 372 ms
Wall time: 30.6 s


### 7.3 - Inspect Generated output

In [21]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.head(10)

Input data dimensions (rows x columns)=  (5, 15)
Output data dimensions (rows x columns)=  (5, 17)
Duplicate chunks removed  by fuzzy-dedupe:   0


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,int_id_column,hash_column
0,mars.pdf,1,0,11,32038943-af7b-43fd-b5e9-c640fdb37c54,pdf,5556be7a44be95d5f40a5235fe1e8774367a070b20eef6...,2505,2024-09-06T01:46:03.416974,1.689363,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is...",$.main-text[5],1,"[132.86, 500.93, 477.3, 534.53]",4,-1
1,mars.pdf,1,0,11,32038943-af7b-43fd-b5e9-c640fdb37c54,pdf,5556be7a44be95d5f40a5235fe1e8774367a070b20eef6...,2505,2024-09-06T01:46:03.416974,1.689363,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[7],1,"[147.72, 452.7, 476.98, 474.97]",5,-1
2,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.82, 590.31, 478.58, 623.05]",1,-1
3,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.89, 512.46, 477.42, 534.47]",2,-1
4,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Earth\n· Distance from the Sun: Average of 149...,$.main-text[7],1,"[147.68, 464.66, 477.03, 487.02]",3,-1


## Document Quality - TODO

## Step-8:   Text encoding

Encode text for the vector storage.

### 8.1 - Set Input/output Folder

In [22]:
STAGE  += 1
# STAGE  = 7 ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_encoder_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output/05_fdedupe_out' --> output='output/06_encoder_out'


### 8.2 - Execute

In [23]:
%%time 

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

01:47:34 INFO - Running locally
01:47:34 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
01:47:34 INFO - data factory data_ is using local data access: input_folder - output/05_fdedupe_out output_folder - output/06_encoder_out
01:47:34 INFO - data factory data_ max_files -1, n_sample -1
01:47:34 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
01:47:34 INFO - pipeline id pipeline_id
01:47:34 INFO - code location None
01:47:34 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
01:47:34 INFO - actor creation delay 0
01:47:34 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
2024-09-06 01:47:36,367	INFO worker.py:1744 -- Started a local Ray instance. View th

✅ Stage:6 completed successfully
CPU times: user 576 ms, sys: 200 ms, total: 776 ms
Wall time: 21.3 s


### 8.3 - Inspect Generated output

You will see a column called `embeddings` added at the end.  This the text content converted into vectors or embeddings.

In [24]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(3)

Input data dimensions (rows x columns)=  (5, 17)
Output data dimensions (rows x columns)=  (5, 18)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,int_id_column,hash_column,embeddings
3,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.89, 512.46, 477.42, 534.47]",2,-1,"[0.0724358, -0.058001805, -0.01977186, -0.0243..."
2,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.82, 590.31, 478.58, 623.05]",1,-1,"[0.0077404897, -0.020559434, 0.026426662, 0.01..."
4,earth.pdf,1,0,11,818d4b3f-f3b7-4453-9c5d-027a72c917bf,pdf,e88caa778725418ac47c79ef03454451a4ce7839361738...,2383,2024-09-06T01:46:03.087877,1.674357,earth.pdf,Earth\n· Distance from the Sun: Average of 149...,$.main-text[7],1,"[147.68, 464.66, 477.03, 487.02]",3,-1,"[0.06319313, 0.05406263, 0.042939227, 0.002394..."


## Step-9: Copy output to final output dir

In [25]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/06_encoder_out' --> 'output/output_final'
